In [1]:
import numpy as np
import pickle 
import sys 
sys.path.append('../')
from simulator.envs import *

In [2]:
with open('../data/DiDi_day1_grid121.pkl', 'rb') as handle:
    data_param = pickle.load(handle) 
dura_param = data_param['duration'] # 每
price_param = data_param['price']   # 每级邻居的价格    第0级表示自己网格 0~l_max
neighbor = data_param['neighbor']    # neighbor>=100 表示不可达的订单
order_param = data_param['order']   # shape=(11,11,144)  表示 (出发地，目的地，出发时间)
l_max = 8   # 最大通勤跨邻居数
M,N = 11,11

In [3]:
order_source = np.sum(order_param,axis=1)
order_source.sum(-1)

array([  135,    23,    13,    30,    58,    47,    41,     4,   195,
         184,    45,    65,   161,   256,   289,  1136,  2171,   496,
         140,   147,    50,     6,    60,   222,   193,   391,   835,
         460,    78,   449,   145,   339,    18,    12,   304,   993,
         938,  6279,  3685,  3723,  2964,   682,   296,   621,   211,
         172,   704,  2770,  8242,  6002, 10936,  4694,  1404,   281,
         938,   232,   927,  1672,  2369,  6598, 12528, 18976, 12167,
        3702,  1120,   238,    72,   327,   983,  2058,  2831, 13054,
        6845,  7241,  1931,   581,   256,    50,   271,   361,  1099,
        1225,  2967,  1805,  2161,   849,  1689,   234,    88,   162,
         175,   136,   353,  1374,   699,   888,   384,   426,    27,
         472,    68,   311,   102,   142,   781,    18,   215,    24,
         154,    47,     4,   119,    68,   105,   237,   278,   257,
          44,     2,    12,     4])

In [4]:
np.where(order_source.sum(-1)>10000)

(array([50, 60, 61, 62, 71]),)

In [12]:
np.where(order_source[61]>200)

(array([ 53,  59,  65,  70,  71,  77,  83,  86,  88,  89,  94,  95,  99,
        100, 101, 105, 106, 107, 109, 112, 113, 115, 116, 118, 119, 125,
        131]),)

In [15]:
order_num = np.sum(order_param, axis=1, keepdims=True)
order_num[order_num==0]=1
order_prob = order_param/ order_num


In [19]:
order_prob[61,:,107]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00641026, 0.00641026, 0.        , 0.        ,
       0.        , 0.        , 0.00641026, 0.00320513, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00641026,
       0.        , 0.00641026, 0.03205128, 0.01282051, 0.01282051,
       0.01282051, 0.00641026, 0.        , 0.00641026, 0.        ,
       0.        , 0.        , 0.01282051, 0.01923077, 0.00641026,
       0.04487179, 0.00641026, 0.01282051, 0.        , 0.00641026,
       0.        , 0.00641026, 0.01923077, 0.00641026, 0.03846154,
       0.08012821, 0.04807692, 0.10897436, 0.03205128, 0.01602564,
       0.        , 0.        , 0.        , 0.00641026, 0.01923077,
       0.02564103, 0.02564103, 0.07692308, 0.06089744, 0.00641

In [44]:
order_num = np.sum(order_param, axis=1)
order_num[order_num>=200]=200
order_num[ np.logical_and(order_num>=1, order_num<=8) ] = 1
order_num[order_num>8]=order_num[order_num>8]/8

order_num[61]

array([ 2,  5, 10,  8,  9, 10,  7,  5,  5,  4,  5,  6,  4,  3,  3,  4,  2,
        5,  2,  2,  1,  2,  2,  5,  2,  2,  2,  1,  1,  2,  1,  1,  1,  1,
        1,  2,  1,  1,  2,  2,  3,  3,  3,  6,  6,  8,  9, 17, 13, 15, 13,
       16, 18, 25, 12, 13, 15, 14, 17, 25, 18, 14, 17, 17, 23, 25, 17, 19,
       15, 19, 25, 25, 21, 20, 16, 19, 22, 25, 18, 19, 20, 21, 23, 25, 24,
       22, 25, 23, 25, 25, 24, 23, 21, 24, 25, 25, 21, 22, 20, 25, 25, 25,
       23, 22, 23, 25, 25, 25, 23, 25, 17, 20, 25, 25, 22, 25, 25, 23, 25,
       25, 20, 19, 22, 22, 21, 25, 20, 19, 18, 16, 22, 25, 15, 18, 13, 12,
       17, 18, 14, 11, 11, 10,  9,  9])

In [46]:
new_order_param = order_num*order_prob
(new_order_param.sum(-1).sum(-1)-new_order_param.sum(0).sum(-1)).astype(np.int32)

array([   624,    173,     72,    149,    225,    172,    156,     48,
          513,    593,    221,    411,    630,    726,    591,    693,
          559,    758,    500,    386,    281,     49,    447,    760,
          788,    641,    609,    612,    255,    548,    379,    652,
           84,     85,    925,    911,    909,  -1967,   -211,   -137,
          -87,    651,    468,    826,    722,    759,   1020,    591,
        -3397,  -2856,  -8454,   -773,    771,    617,    797,    860,
          971,   1029,    913,  -2069,  -9075, -21960,  -8452,    -10,
          894,    763,    539,   1013,   1093,   1107,    943, -10938,
         -653,  -2140,    926,   1035,    815,    441,    956,   1073,
         1187,   1214,    877,   1201,   1056,   1150,    966,   1036,
          572,    849,    861,    804,   1108,   1246,   1233,   1196,
         1163,   1106,    266,   1315,    517,   1120,    756,    816,
         1274,    203,   1008,    191,    963,    413,     70,    728,
      

In [47]:
new_order_param.astype(np.int32).sum(-1).sum(-1)

array([ 607,  174,   73,  148,  218,  171,  155,   48,  482,  559,  220,
        406,  605,  674,  536,  480,  361,  644,  480,  362,  280,   50,
        445,  725,  748,  559,  445,  510,  252,  463,  364,  586,   84,
         85,  853,  687,  698,  193,  166,  207,  247,  535,  405,  685,
        676,  732,  852,  442,  278,  191,  201,  333,  606,  561,  595,
        811,  745,  724,  631,  335,  248,  226,  298,  399,  701,  715,
        532,  944,  885,  789,  785,  256,  607,  385,  679,  910,  763,
        439,  901,  984,  958,  958,  734,  904,  799,  955,  740,  991,
        560,  818,  830,  784, 1033,  973, 1075,  985, 1087, 1002,  264,
       1202,  514, 1060,  741,  797, 1095,  204,  975,  190,  939,  407,
         70,  712,  524,  610,  995, 1019, 1052,  383,   24,  114,   37])

In [48]:
new_order_param.astype(np.int32).sum(-1).sum(0)

array([    0,     0,     0,     0,     0,     0,     0,     0,     1,
           0,     0,     0,     1,     0,     3,     7,    45,     6,
           0,     0,     1,     0,     0,     2,     1,     1,     8,
           2,     0,     5,     2,     2,     0,     0,     1,     7,
           7,  1731,   414,   378,   413,    12,     1,     1,     0,
           1,     3,   106,  2894,  2470,  7447,   925,    20,     3,
           4,     0,     2,    13,    46,  1837,  8155, 20674,  7640,
         378,    10,     2,     0,     2,     8,    17,    93, 10037,
         918,  1979,    22,     5,     0,     0,     0,     0,     6,
           3,    89,    15,    30,     3,    24,     0,     0,     0,
           0,     0,     1,     4,     2,     3,     0,     2,     0,
           0,     0,     1,     0,     0,     1,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     1,     0,
           0,     0,     0,     0])

In [8]:
order_param1 = order_param.copy()
index = order_param1.sum(axis=1)>200
index = np.stack([index]*121,axis=1)
order_param1[index] = order_param1[index]/2

In [14]:
index[61,:,107]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])